In [1]:
import pandas as pd
import openai

df = pd.read_csv("selected_kk_context.csv")

In [3]:
df

,Unnamed: 0,id,url,title,text
0,0,1030,https://kk.wikipedia.org/wiki/%D2%9A%D0%B0%D0%...,Қазақстан,"Қазақстан ( ), толық атауы Қазақстан Республик..."
1,1,1032,https://kk.wikipedia.org/wiki/%D0%A8%D1%8B%D0%...,Шымкент,"Шымкент — Қазақстандағы қала, Республикалық ма..."
2,2,1035,https://kk.wikipedia.org/wiki/%D2%9A%D1%8B%D1%...,Қырғызстан,"Қырғызстан (), ресми Қырғыз Республикасы () — ..."
3,3,1471,https://kk.wikipedia.org/wiki/%D2%9A%D0%B0%D1%...,Қарағанды,"Қарағанды — Орталық Қазақстандағы қала, Қараға..."
4,4,1650,https://kk.wikipedia.org/wiki/%D0%90%D2%9B%D0%...,Ақмола облысы,Ақмола облысы — Қазақстан Республикасының солт...
...,...,...,...,...,...
1189,1189,712000,https://kk.wikipedia.org/wiki/%D0%9C%D1%96%D1%...,"Міржақып. Оян, Қазақ (фильм, 2023)","«Міржақып. Оян, Қазақ» — Міржақып Дулатұлы тур..."
1190,1190,712072,https://kk.wikipedia.org/wiki/%D0%A5%D0%B0%D0%...,Халықтар Достығы ордені (Беларусь),Халықтар Достығы ордені () — Беларусь мемлекет...
1191,1191,713062,https://kk.wikipedia.org/wiki/%D0%A2%D3%A9%D0%...,Төленді Арыстанбеков,"Төленді Арыстанбеков (15 тамыз 1939; Бөрлі, Қо..."
1192,1192,713146,https://kk.wikipedia.org/wiki/%D0%96%D0%B5%D0%...,Жекпе-жек (батырлар айқасы),Жекпе-жек – майдандағы үлкен шайқас алдында кә...


In [6]:
text = df['text'].tolist()

In [13]:
len(text)

1194

In [2]:
import os
from openai import OpenAI

api_key = ""

client = OpenAI(
    api_key=api_key,
)

In [34]:
# STOP

from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="WHY_QS", description="A 'why' question in kazakh language"),
    ResponseSchema(name="WHAT_QS", description="A 'what' question in kazakh language"),
    ResponseSchema(name="HOW_QS", description="A 'how' question in kazakh language"),
    ResponseSchema(name="DESCRIBE_QS", description="A 'describe' question in kazakh language"),
    ResponseSchema(name="ANALYZE_QS", description="A 'analyze' question in kazakh language"),
]

system_prompt = """
Based on the text provided below, generate five diverse and challenging questions in the Kazakh language. 
Ensure that the questions are general and not overly specific, avoiding details such as specific years or unique facts.
The questions should assess comprehension of the text and follow this format:

1. A "why" question
2. A "what" question
3. A "how" question
4. A "describe" question
5. An "analyze" question

Reiterating: the questions that you will create based on the context below should be ONLY and STRICTLY BE IN KAZAKH LANGUAGE.

STRICTLY FOLLOW FORMATTING BELOW:
"""

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

formatted_system_prompt = f"{system_prompt}\n\n{format_instructions}"

In [35]:
print(formatted_system_prompt)


Based on the text provided below, generate five diverse and challenging questions in the Kazakh language. 
Ensure that the questions are general and not overly specific, avoiding details such as specific years or unique facts.
The questions should assess comprehension of the text and follow this format:

1. A "why" question
2. A "what" question
3. A "how" question
4. A "describe" question
5. An "analyze" question

Reiterating: the questions that you will create based on the context below should be ONLY and STRICTLY BE IN KAZAKH LANGUAGE.

STRICTLY FOLLOW FORMATTING BELOW:


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"WHY_QS": string  // A 'why' question in kazakh language
	"WHAT_QS": string  // A 'what' question in kazakh language
	"HOW_QS": string  // A 'how' question in kazakh language
	"DESCRIBE_QS": string  // A 'describe' question in kazakh language
	"ANALYZE_QS": string  // A

In [22]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": formatted_system_prompt+text[0],
        }
    ],
    model="gpt-4o-mini-2024-07-18",
)

In [23]:
response_text = chat_completion.choices[0].message.content

In [24]:
print(response_text)

```json
{
	"why_qs": "Қазақстанның геосаяси жағдайы неге маңызды?",
	"what_qs": "Қазақстанның табиғи ресурстарының қандай түрлері бар?",
	"how_qs": "Қазақстан Республикасында саяси партиялар қалай құрылуға болады?",
	"describe_qs": "Қазақстанның климаттық ерекшеліктерін сипаттаңыз.",
	"analyze_qs": "Қазақстанның тәуелсіздік алғаннан кейінгі экономикалық дамуын талдаңыз."
}
```


In [26]:
parsed_response = output_parser.parse(response_text)

In [27]:
parsed_response

{'why_qs': 'Қазақстанның геосаяси жағдайы неге маңызды?',
 'what_qs': 'Қазақстанның табиғи ресурстарының қандай түрлері бар?',
 'how_qs': 'Қазақстан Республикасында саяси партиялар қалай құрылуға болады?',
 'describe_qs': 'Қазақстанның климаттық ерекшеліктерін сипаттаңыз.',
 'analyze_qs': 'Қазақстанның тәуелсіздік алғаннан кейінгі экономикалық дамуын талдаңыз.'}

# LETS TRY SUKA

In [28]:
random_entries = df.sample(n=20, random_state=42069)

In [29]:
random_entries

,Unnamed: 0,id,url,title,text
787,787,117804,https://kk.wikipedia.org/wiki/%D0%91%D0%B5%D1%...,Бес тас (ойын),"«Бестас» — өте көне дәуірден келе жатқан, қаза..."
315,315,29394,https://kk.wikipedia.org/wiki/%D2%9A%D0%B0%D0%...,Қанжыға,Қанжыға – ерге бөктерген затты байлайтын қайыс...
41,41,4930,https://kk.wikipedia.org/wiki/%D0%96%D2%B1%D0%...,Жұмекен Сабырұлы Нәжімеденов,"Жұмекен Сабырұлы Нәжімеденов (28 қараша, 1935 ..."
600,600,70965,https://kk.wikipedia.org/wiki/%D0%A0%D0%B5%D1%...,Ресей мәдениеті,Ресей мәдениеті — адамзат баласына баға жетпес...
79,79,7469,https://kk.wikipedia.org/wiki/%D2%9A%D0%BE%D1%...,Қошмұхамбет Дүйсебайұлы Кемеңгеров,Қошмұхамбет Дүйсебайұлы Кемеңгеров (15 шілде 1...
1037,1037,536397,https://kk.wikipedia.org/wiki/%D0%A5%D0%B0%D0%...,Хамит Ерғалиев ауылы,Хамит Ерғалиев ауылы (1998 ж. дейін - Новобога...
198,198,20585,https://kk.wikipedia.org/wiki/%D0%A8%D0%BE%D1%...,Шотан батыр,ШОТАН БАТЫР\n\nЖалпы ақпарат\nШамамен 18 ғасыр...
28,28,3428,https://kk.wikipedia.org/wiki/%D0%9E%D1%80%D0%...,Орал,Орал (; 1775 жылға шейін - Яицкий қалашығы) — ...
1164,1164,695508,https://kk.wikipedia.org/wiki/%D0%91%D1%96%D0%...,Білім-қазына қоғамдық қоры,Білім-қазына қоғамдық қоры 2015 жылы Нұр-Мүбар...
574,574,68890,https://kk.wikipedia.org/wiki/%D0%AD%D1%81%D1%...,Эстетикалық көзқарасы,Эстетикалық көзқарасы. Абай Құнанбаевтың арнай...


In [31]:
random_text = random_entries['text'].tolist()
random_title = random_entries['title'].tolist()

In [36]:
results = []

for idx, c_text in enumerate(random_text):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                    {
                        "role": "user",
                        "content": formatted_system_prompt+c_text,
                    }
                ],
                model="gpt-4o-mini-2024-07-18",
            )
        
        response_text = chat_completion.choices[0].message.content
        parsed_response = output_parser.parse(response_text)
        results.append((random_title[idx], parsed_response))

    except Exception as e:
        print(e)
        print('pohui')

Got invalid return object. Expected key `WHAT_QS` to be present, but got {'WHY_QS': 'Нұр-Мүбарак университеті мен Білім-қазына қорының ашылуының негізгі себептері қандай?', 'What_QS': 'Білім-қазына қоғамдық қорының негізгі мақсаты неде?', 'HOW_QS': 'Білім-қазына қоры университеттің студенттеріне қандай әлеуметтік-материалдық қолдау көрсетеді?', 'DESCRIBE_QS': 'Нұр-Мүбарак университетінің тарихы мен жетістіктерін қалай сипаттауға болады?', 'ANALYZE_QS': 'Білім-қазына қоғамдық қорының қаржылары мен жұмсау бағыттарын қалай талдауға болады?'}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
pohui


In [37]:
results

[('Бес тас (ойын)',
  {'WHY_QS': 'Бестас ойынының балаларға тигізетін әсері неден көрінеді?',
   'WHAT_QS': 'Бестас ойынының негізгі ережелері қандай?',
   'HOW_QS': 'Бестас ойынында ойыншылардың кезек алу тәсілі қалай жүзеге асырылады?',
   'DESCRIBE_QS': 'Бестас ойынының кезеңдерін сипаттап беріңіз.',
   'ANALYZE_QS': 'Бестас ойынының қоғамдағы мәдени және тәрбиелік мәнін талқылаңыз.'}),
 ('Қанжыға',
  {'WHY_QS': 'Қанжығалау салтының маңызы неде?',
   'WHAT_QS': 'Қанжығалау процесі қалай жүзеге асырылады?',
   'HOW_QS': 'Қанжығаға аң байлау кезінде қандай дәстүрлер мен ырымдар бар?',
   'DESCRIBE_QS': 'Қанжыға мен қанжығалау құбылыстарын сипаттап беріңіз.',
   'ANALYZE_QS': 'Аңшылар мен жолаушылар арасындағы қарым-қатынасты қанжыға арқылы қалай талдауға болады?'}),
 ('Жұмекен Сабырұлы Нәжімеденов',
  {'WHY_QS': 'Жұмекен Нәжімеденовтің шығармашылығында ел мен жер тақырыбы не себепті басты орын алады?',
   'WHAT_QS': 'Жұмекен Нәжімеденовтің қандай маңызды шығармалары бар?',
   'HOW_QS'